In [1]:
import pandas as pd
import re

import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import warnings; warnings.simplefilter('ignore')

In [2]:
def full_scraper_lite(n_books=5): 
    
    def scrap_anagrama():

        def get_links_colecciones():
            '''
            Devuelve diccionario que contiene las colecciones
            de la editorial Anagrama
            '''
            html = requests.get(url_colecciones, headers).content
            soup = BeautifulSoup(html)
            colecciones = {}
            for link in soup.findAll('a'):
                coleccion_link = url_anagrama + str(link.get('href'))
                coleccion = coleccion_link.rsplit('/', 1)[-1].replace('-',' ')
                if "/coleccion/" in coleccion_link:
                    colecciones[coleccion] = coleccion_link
            del colecciones['ebooks']
            return colecciones


        def get_links_libros_in_page(url_page):
            '''
            Devuelve una lista que contiene los libros
            de una página del sitio web
            '''
            html = requests.get(url_page, headers).content
            soup = BeautifulSoup(html)
            links_libros = set([link['href'] for link in soup.find_all(
                "a", class_="book", href=True) if link['href'].startswith('/libro/')])
            return links_libros


        def get_libro_info(url_libro):
            '''
            Recoge la información básica del libro
            dada una url de la editorial Anagrama
            y lo carga en un diccionario
            '''
            url_libro = url_libro
            html = requests.get(url_libro, headers).content
            soup = BeautifulSoup(html)

            datos_libro = dict()

            # Básicos
            datos_libro["TÍTULO"] = soup.h1.get_text()
            datos_libro["AUTOR/A"] = soup.title.get_text().split('-')[1].strip()
            datos_libro["RESUMEN"] = soup.find('div', class_="textContent mt10px").get_text()

            # Tabla
            for e in soup.find('table'):
                data = e.find_all('td')
                col, row = data[0].get_text(), data[1].get_text()
                datos_libro[col] = row

            # Ebook 
            datos_libro["EBOOK"] = soup.find("div", class_="tab-pane ebook") != None

            # Corrección de la información de COLECCIÓN
            datos_libro["COLECCIÓN"] = url_libro.split('/')[4].replace('-',' ').title()


            return datos_libro


        def get_row_values(info_libro):
            '''
            Devuelve una lista de la información de un libro en formato apto
            para ser cargado en la base de datos
            '''
            columns = ['ISBN', 'EAN', 'TÍTULO', 'AUTOR/A', 'COLECCIÓN', 'CÓDIGO',
                       'NÚM. DE PÁGINAS', 'PUBLICACIÓN', 'PVP CON IVA', 'RESUMEN',
                       'TRADUCCIÓN', 'EBOOK']
            for key, value in info_libro.items():
                row_values = []
                for column in columns:
                    try:
                        row_values.append(info_libro[column])
                    except:
                        row_values.append(None)
            return row_values

        url_anagrama = 'https://www.anagrama-ed.es'
        url_colecciones = 'https://www.anagrama-ed.es/colecciones'

        headers = {"User_Agent":UserAgent().random}  # User agent generado aleatoriamente

        colecciones = get_links_colecciones()

        for coleccion, link in colecciones.items():
            page = 1
            while bool(page)==True:
                url_page = link + '?p=' + str(page)
                listado_links_libros = get_links_libros_in_page(url_page)
                if len(listado_links_libros) != 0:
                    for url_libro in listado_links_libros:
                        url_libro = url_anagrama + url_libro
                        info_libro = get_libro_info(url_libro)
                        row_values = get_row_values(info_libro)
                        rows.append(row_values)

                        if len(rows) == n_books:
                            return rows
                    page += 1
                else:
                    page = False
    
    
    def scrap_Amazon_sales(ean):
        '''
        Toma el ean de un libro y devuelve su posición
        en distintos ranking de ventas de Amazon
        '''
        # Ajustes del driver
        options = Options()
        options.add_argument('--headless')
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        wait = WebDriverWait(driver, 10)                 
        driver.delete_all_cookies()

        # Acceso al artículo de Amazon
        amazon_searcher = "https://www.amazon.com/advanced-search/books"
        driver.get(amazon_searcher)

        search_isbn = driver.find_element(By.XPATH, '//*[(@id = "field-isbn")]')
        search_isbn.send_keys(str(ean))
        wait.until(EC.element_to_be_clickable(search_isbn)).submit()

        wait.until(EC.url_changes(amazon_searcher))

        wait.until(EC.element_to_be_clickable((By.XPATH,
                                              '//*[contains(concat( " ", @class, " " ), concat( " ", "s-line-clamp-2", " " ))]')
                                             )).click()

        # Extracción información ventas
        soup = BeautifulSoup(driver.page_source)
        try: 
            top_sales = soup.find("div", {"id": "detailBulletsWrapper_feature_div"}).find_all(
            "ul", class_= "a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list")[1].get_text()
        except:
            top_sales = None

        # Cierre del driver
        driver.quit()

        return (ean, top_sales)
    
    
    columns = ['ISBN', 'EAN', 'TÍTULO', 'AUTOR/A', 'COLECCIÓN', 'CÓDIGO',
                   'NÚM. DE PÁGINAS', 'PUBLICACIÓN', 'PVP CON IVA', 'RESUMEN',
                   'TRADUCCIÓN', 'EBOOK']
    rows = []
    
    scrap_anagrama()
    
    df =  pd.DataFrame(rows, columns=columns)
    
    eans = df['EAN'].to_list()
    top_sales_by_ean = []
    for ean in eans:
        try:
            top_sales_by_ean.append(scrap_Amazon_sales(ean))
        except:
            continue
            
    d_tops = dict()
    for ean, top in top_sales_by_ean:
        if top != None and top != '   ':
            tops_libro = top.strip().split('nº')[1:]
            for i, t in enumerate(tops_libro):
                t = re.sub(r'\(Ver el Top 100.*\)', '', t)
                tops_libro[i] = t.strip()
            d_tops[ean] = tops_libro

    df_amazon_tops = pd.DataFrame([d_tops.keys(), d_tops.values()]).transpose()
    df_amazon_tops.columns = ['EAN', 'TOPS_AMAZON']

    return pd.merge(df,df_amazon_tops,on='EAN',how='left')

In [4]:
full_scraper_lite(n_books=3)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [C:\Users\Usuario\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache


====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [C:\Users\Usuario\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache


====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [C:\Users\Usuario\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache


,ISBN,EAN,TÍTULO,AUTOR/A,COLECCIÓN,CÓDIGO,NÚM. DE PÁGINAS,PUBLICACIÓN,PVP CON IVA,RESUMEN,TRADUCCIÓN,EBOOK,TOPS_AMAZON
0,978-84-339-8110-3,9788433981103,Sed,"Nothomb, Amélie",Panorama De Narrativas,PN 1070,128,02/02/2022,17.9 €,Una apasionante y nothombiana reelaboración de...,Sergi Pàmies,True,"[473,559 en Tienda Kindle, 1,244 en Literatura..."
1,978-84-339-8112-7,9788433981127,"Bueno, aquí estamos","Swift, Graham",Panorama De Narrativas,PN 1072,184,16/02/2022,18.9 €,Una novela de gran sensibilidad que nos envuel...,Antonio-Prometeo Moya Valle,True,NaN
2,978-84-339-8111-0,9788433981110,Nacido de ninguna mujer,"Bouysse, Franck",Panorama De Narrativas,PN 1071,304,02/02/2022,19.9 €,Un manuscrito. Una joven enfrentada a un desti...,Rosa Alapont Calderaro,True,"[2,217,020 en Libros, 104,531 en Ficción Liter..."
